In [ ]:
# dont_test
# default_exp scripts
%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
# export

import re
import sys

from nbdev.export2html import nbdev_build_docs


def nbdev_build_docs_patched():
    print("hello world")
    sys.argv[0] = re.sub(r"(-script\.pyw|\.exe)?$", "", sys.argv[0])
    sys.exit(nbdev_build_docs())

<IPython.core.display.Javascript object>